# Wissensaggregator Mittelalter und frühe Neuzeit

WIAGweb2  
Daten für Utrechter Priester aus Excel einlesen

[Neuen Typ anlegen](#Neuen-Typ-anlegen)  
[Literatur](#Literatur)    
Priester  
- [Einträge in `item`](#Einträge-in-item)
- [Daten eintragen](#Daten-eintragen)
- [Freie Attribute](#Freie-Attribute)
- [Literaturverweise eintragen](#Literaturverweise-eintragen)
- [Namensvarianten eintragen](#Namensvarianten-eintragen)
- [Suchvarianten eintragen (`name_lookup`)](#Suchvarianten-eintragen-in-name_lookup)

[Ämter](#Ämter)

## Vorspann

Modul laden

In [2]:
wds_path = "../.."

"../.."

In [3]:
cd(wds_path)

In [4]:
using Pkg

In [5]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [6]:
using Revise

In [7]:
using WiagDataSetup

In [8]:
Wds = WiagDataSetup

WiagDataSetup

In [9]:
Wds.setDBWIAG(user = "georg", db = "wiag2")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2")

In [10]:
using DataFrames, CSV, Dates

In [11]:
ENV["COLUMNS"] = 120 # Anzeige/Ausgabe für die Inhalte von DataFrames vergrößern

120

In [12]:
computer = :bkroege_goe
computer = :ghertko_kn
if computer == :bkroege_goe
    data_path = "../data"
    out_path = "../data_sql"
else
    data_path = "C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data"
    out_path = "C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data_sql"
end

"C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data_sql"

In [13]:
item_type_id = 10

10

## Daten löschen
z.B. für ein komplettes Update

In [269]:
sql_item_ids = "SELECT ID FROM item WHERE item_type_id = $(item_type_id)"
sql = "
DELETE FROM person_birthplace WHERE person_id in ($(sql_item_ids));
DELETE FROM person_role WHERE person_id in ($(sql_item_ids));
DELETE FROM name_lookup WHERE person_id in ($(sql_item_ids));
DELETE FROM person WHERE id in ($(sql_item_ids));
DELETE FROM item_property WHERE item_id in ($(sql_item_ids));
DELETE FROM item_reference WHERE item_id in ($(sql_item_ids));
DELETE FROM item WHERE item_type_id = $(item_type_id);"
file_name = joinpath(out_path, "delete_item_type_$(item_type_id).sql")
open(file_name, "w") do io
    println(io, sql)
end

## Neuen Typ anlegen

(einmalig)

einlesen aus CSV

In [71]:
file_name = joinpath(data_path, "item_type.csv");
df_item_type = CSV.read(file_name, DataFrame, delim = ';')

,id,comment,name,note,created_by,date_created,changed_by,date_changed
,Int64,Missing,String31,String,Int64,String31,Int64,String31
1,10,missing,Priester Utrecht,Priester des Bistums Utrecht nach Stapel,2,29.03.2022 14:00,2,29.03.2022 14:00


In [ ]:
create_date(date_time) = typeof(date_time) == DateTime ? date_time : DateTime(date_time, dateformat"dd.mm.yy HH:MM")

In [ ]:
transform!(df_item_type, :date_created => ByRow(create_date) => :date_created);

In [ ]:
transform!(df_item_type, :date_changed => ByRow(create_date) => :date_changed);

In [ ]:
names(df_item_type)

alternativ: direkt erstellen

In [ ]:
df_item_type = DataFrame();

In [ ]:
insertcols!(df_item_type,
    :id => item_type_id,
    :comment => missing,
    :name => "Priester Utrecht",
    :note => "Priester des Bistums Utrecht nach Stapel",
    :table_name => "person",
    :id_public_key => "Pers-PRIESTUtrecht",
    :name_app => "priest_ut",
    :created_by => 2,
    :date_created => now(),
    :changed_by => 2,
    :date_changed => now(),
)

In [ ]:
file_name = joinpath(out_path, "item_type.sql")
Wds.insert_sql(file_name, "item_type", df_item_type)

## Literatur

In [14]:
df_reference_volume = DataFrame();

In [15]:
insertcols!(df_reference_volume,
    :id => missing, # wird automatisch vergeben
    :comment => missing,
    :full_citation => ["Stapel, R.J. (2020) Ordinations of priests in the medieval diocese of Utrecht (1505-1518). IISH Data Collection.",
        "Brom, Gisbert, ''Naamlijst der priesters, die in het bisdom Utrecht gewijd zijn van 1505-1518'', Archief voor het Aartsbisdom Utrecht 23 (1896), pp. 386-471",
        "Brom, Gisbert, ''Naamlijst der priesters, die in het bisdom Utrecht gewijd zijn van 1505-1518 (vervolg)'', Archief voor het Aartsbisdom Utrecht 24 (1897), pp. 1-85",
        ],
    :title_short => ["Stapel, Ordinations of priests in the medieval diocese of Utrecht (1505-1518)",
        "G. Brom, ''Naamlijst der priesters…'', AAU 23 (1896)",
        "G. Brom, ''Naamlijst der priesters…'', AAU 24 (1897)"],
    :author_editor => ["Rombert Stapel", "Gisbert Brom", "Gisbert Brom"],
    :note => missing,
    :ri_opac_id => missing,
    :isbn => missing,
    :year_publication => ["2020", "1896", "1897"],
    :display_order => [1, 2, 3],
    :gs_volume_nr => missing,
    :gs_doi => missing,
    :gs_url => missing,
    :online_resource => ["https://hdl.handle.net/10622/IEYIUX", missing, missing],
    :reference_id => [1, 2, 3],
    :item_type_id => item_type_id
)

,id,comment,full_citation
,Missing,Missing,String
1,missing,missing,"Stapel, R.J. (2020) Ordinations of priests in the medieval diocese of Utrecht (1505-1518). IISH Data Collection."
2,missing,missing,"Brom, Gisbert, ''Naamlijst der priesters, die in het bisdom Utrecht gewijd zijn van 1505-1518'', Archief voor het Aartsbisdom Utrecht 23 (1896), pp. 386-471"
3,missing,missing,"Brom, Gisbert, ''Naamlijst der priesters, die in het bisdom Utrecht gewijd zijn van 1505-1518 (vervolg)'', Archief voor het Aartsbisdom Utrecht 24 (1897), pp. 1-85"


In [16]:
file_name = joinpath(out_path, "reference_volume.sql")
Wds.insert_sql(file_name, "reference_volume", df_reference_volume)

3

## Einträge in `item`

Daten für die Priester einlesen.

Exportiere die Daten aus Excel über "Speichern unter" im Format "CSV UTF-8" (Auswahlfeld Dateityp)

*2022-05-30* Verwende `Ordinations_of_priests_Stapel.csv` statt `Ordinations_of_priests_Stapel_BK.csv`

In [251]:
file_name = joinpath(data_path, "Ordinations_of_priests_Stapel_cs1.csv");
df_priests = CSV.read(file_name, DataFrame, delim = '\t', stringtype = String);

In [252]:
Wds.clean_up!(df_priests)

In [253]:
size(df_priests, 1)

4643

In [254]:
df_priests[41:45, [:Bron, :Transcriptie]]

,Bron,Transcriptie
,String,String
1,"G. Brom, 'Naamlijst der priesters…', AAU 23/24 (1896/1897), p. 390",† Frater Judocus Gerardi ordinis sancti Augustini
2,"G. Brom, 'Naamlijst der priesters…', AAU 23/24 (1896/1897), p. 390",Johannes Jacobi de Campis
3,"G. Brom, 'Naamlijst der priesters…', AAU 23/24 (1896/1897), p. 390",Amelius Laurencii de Assendelft
4,"G. Brom, 'Naamlijst der priesters…', AAU 23/24 (1896/1897), p. 390",Petrus Anthonii de Corthenis
5,"G. Brom, 'Naamlijst der priesters…', AAU 23/24 (1896/1897), p. 390",Cornelius Wilhelmi Bloemmert


In [255]:
df_priests = unique(df_priests);

Der Datensatz einhält Zeilen, die in allen Feldern übereinstimmen.

In [55]:
columns = [
    :Bron,
    :Seite,
    Symbol("Type record"),
    :Priester_ID,
    Symbol("Datum (YMD)"),
    :Locatie,
    :Transcriptie,
    :Voetnoot_bron,
    :Bijdrage,
    :Titel,
    :Voornaam,
    :Patroniem,
    :Familienaam,
    Symbol("Familienaam, toponiem"),
    Symbol("Familienaam, onzeker"),
    :Toponiem,
    :Orde,
    :Functie,
    :Instelling,
];

In [256]:
dropmissing!(df_priests, :Priester_ID);

In [257]:
df_priests = unique(df_priests);

In [258]:
size(df_priests)

(4477, 28)

In [ ]:
names(df_priests)

In [62]:
columns = [
    :Priester_ID, 
    Symbol("Datum (YMD)"),
    :Titel, 
    :Voornaam,
    :Familienaam,    
]

5-element Vector{Symbol}:
 :Priester_ID
 Symbol("Datum (YMD)")
 :Titel
 :Voornaam
 :Familienaam

In [63]:
df_priests[100:105, columns]

,Priester_ID,Datum (YMD),Titel,Voornaam,Familienaam
,String,Date,String?,String?,String?
1,id_1505_3_17,1505-03-22,missing,Martinus,missing
2,id_1505_3_18,1505-03-22,missing,Wilhelmus,missing
3,id_1505_3_19,1505-03-22,missing,Martinus,missing
4,id_1505_3_20,1505-03-22,missing,Jeronimus,missing
5,id_1505_3_21,1505-03-22,missing,Wilhelmus,missing
6,id_1505_3_22,1505-03-22,missing,Bartoldus,missing


In [259]:
columns = [
    :Priester_ID => :id_in_source,
    :Transcriptie => :item_in_source,
]

2-element Vector{Pair{Symbol, Symbol}}:
  :Priester_ID => :id_in_source
 :Transcriptie => :item_in_source

In [260]:
df_item = select(df_priests, columns);

Ein vorangestelltes `†` bedeutet, dass ein verringerter Beitrag geleistet wurde. Für den Eintrag in `item_in_source` soll es hier nicht weiter enthalten sein.

In [261]:
lstrip_bijdrage(s) = ismissing(s) ? s : lstrip(s, ['†', ' '])

lstrip_bijdrage (generic function with 1 method)

In [263]:
transform!(df_item, :item_in_source => ByRow(lstrip_bijdrage) => :item_in_source);

Verwende nur den Nummern-Teil der ID für die WIAG-ID

In [67]:
get_id_num(id_long) = replace(id_long, "id_" => "")

get_id_num (generic function with 1 method)

In [68]:
transform!(df_item, :id_in_source => ByRow(get_id_num) => :id_num_in_source);

In [69]:
maximum(length.(df_item.id_num_in_source))

10

*2022-05-31* id_public_key = "Pers-PRIESTUtrecht"

In [72]:
id_public_key = df_item_type[1, :id_public_key]

"Pers-PRIESTStapel"

In [73]:
function make_id_public(id)        
    num_id_length = 10
    return "WIAG-" * id_public_key * "-" * lpad(id, num_id_length, '0')
end

make_id_public (generic function with 1 method)

In [74]:
transform!(df_item, :id_num_in_source => ByRow(make_id_public) => :id_public);

In [75]:
df_item[200:205, :]

,id_in_source,item_in_source,id_num_in_source,id_public
,String,String,String,String
1,id_1505_4_52,Nanno Petri de Alcmaria,1505_4_52,WIAG-Pers-PRIESTStapel-01505_4_52
2,id_1505_4_53,Cornelius Petri de Brouershaven,1505_4_53,WIAG-Pers-PRIESTStapel-01505_4_53
3,id_1505_4_54,† Wisso Jacobi,1505_4_54,WIAG-Pers-PRIESTStapel-01505_4_54
4,id_1505_4_55,† Wouterus Johannis de Meclinia,1505_4_55,WIAG-Pers-PRIESTStapel-01505_4_55
5,id_1505_4_56,Magister Johannes Blessinck de Steenwijck,1505_4_56,WIAG-Pers-PRIESTStapel-01505_4_56
6,id_1505_4_57,Johannes Wilhelmi de Wesop,1505_4_57,WIAG-Pers-PRIESTStapel-01505_4_57


Der Wert für `lang` ist das Kürzel für die Sprache der Quelle also `nl`?!

In [76]:
insertcols!(df_item,
    :id => missing,
    :item_type_id => item_type_id,
    :comment => missing,
    :merged_into_id => missing,
    :edit_status => "online",
    :created_by => 2,
    :date_created => now(),
    :changed_by => 2,
    :date_changed => now(),
    :is_deleted => 0,
    :normdata_edited_by => missing,
    :lang => "nl",
    :is_online => 1
);

In [77]:
df_item[200:205, :]

,id_in_source,item_in_source,id_num_in_source,id_public,id,item_type_id
,String,String,String,String,Missing,Int64
1,id_1505_4_52,Nanno Petri de Alcmaria,1505_4_52,WIAG-Pers-PRIESTStapel-01505_4_52,missing,10
2,id_1505_4_53,Cornelius Petri de Brouershaven,1505_4_53,WIAG-Pers-PRIESTStapel-01505_4_53,missing,10
3,id_1505_4_54,† Wisso Jacobi,1505_4_54,WIAG-Pers-PRIESTStapel-01505_4_54,missing,10
4,id_1505_4_55,† Wouterus Johannis de Meclinia,1505_4_55,WIAG-Pers-PRIESTStapel-01505_4_55,missing,10
5,id_1505_4_56,Magister Johannes Blessinck de Steenwijck,1505_4_56,WIAG-Pers-PRIESTStapel-01505_4_56,missing,10
6,id_1505_4_57,Johannes Wilhelmi de Wesop,1505_4_57,WIAG-Pers-PRIESTStapel-01505_4_57,missing,10


In [78]:
table_name = "item"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, select(df_item, Not(:id_num_in_source)))

┌ Info: row 
│   i = 2000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603
┌ Info: row 
│   i = 4000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603


4477

begin fix `item_in_source`

In [264]:
sql = "SELECT * FROM item WHERE item_type_id = 10";
df_item = Wds.sql_df(sql);

In [265]:
lstrip_bijdrage(s) = ismissing(s) ? s : lstrip(s, ['†', ' '])

lstrip_bijdrage (generic function with 1 method)

In [266]:
transform!(df_item, :item_in_source => ByRow(lstrip_bijdrage) => :item_in_source);

In [268]:
table_name = "item"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, df_item)

┌ Info: row 
│   i = 2000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603
┌ Info: row 
│   i = 4000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603


4477

end fix

In [ ]:
begin fix id_public

In [302]:
sql = "SELECT * FROM item WHERE item_type_id = 10";
df_item = Wds.sql_df(sql);

In [304]:
set_id_public_prefix(s) = replace(s, "PRIESTStapel" => "PRIESTUtrecht")

set_id_public_prefix (generic function with 1 method)

In [305]:
transform!(df_item, :id_public => ByRow(set_id_public_prefix) => :id_public);

In [306]:
df_item[401:405, [:id, :id_in_source, :item_in_source, :id_public]]

,id,id_in_source,item_in_source,id_public
,Int32,String?,String?,String
1,312727,id_1506_3_43,Magister Jacobus Ghijsberti de Hoern,WIAG-Pers-PRIESTUtrecht-01506_3_43
2,312728,id_1506_3_44,Hermannus Joconis de Scneeckis,WIAG-Pers-PRIESTUtrecht-01506_3_44
3,312729,id_1506_3_45,Johannes Florencii de Hoern,WIAG-Pers-PRIESTUtrecht-01506_3_45
4,312730,id_1506_3_46,Wolterus Wilhelmi de Harlem,WIAG-Pers-PRIESTUtrecht-01506_3_46
5,312731,id_1506_3_47,Johannes Heinrici de Stavenissa,WIAG-Pers-PRIESTUtrecht-01506_3_47


In [307]:
table_name = "item"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, df_item)

┌ Info: row 
│   i = 2000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603
┌ Info: row 
│   i = 4000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603


4477

end fix

Importiere `item.sql` via phpMyAdmin in die Datenbank.  

## Daten eintragen

Exportiere die Tabelle `item` aus der Datenbank via phpMyAdmin mit den Optionen: "Spaltennamen in die erste Zeile setzen" und "Spalten trennen mit ';'" als Trennzeichen.

In [12]:
# Quelle: Server
if computer == :bkroege_goe 
    file_name = joinpath(data_path, "item.csv");
else
    file_name = joinpath(data_path, "item_local.csv")
end
df_item_db = CSV.read(file_name, DataFrame, delim = ';');

In [17]:
# Quelle: Datenbank lokal
sql = "SELECT id, id_in_source, item_type_id
FROM item 
WHERE item_type_id = 10";
df_idx = Wds.sql_df(sql);

In [18]:
size(df_idx)

(4476, 3)

*2022-05-25* lies Daten aus `Ordinations_of_priests_Stapel.csv` statt `Ordinations_of_priests_Stapel_BK.csv`.

In [19]:
file_name = joinpath(data_path, "Ordinations_of_priests_Stapel_cs1.csv");
df_priests = CSV.read(file_name, DataFrame, delim = '\t', stringtype = String);

In [20]:
Wds.clean_up!(df_priests)

In [21]:
dropmissing!(df_priests, :Priester_ID);

In [22]:
df_priests = unique(df_priests);

In [23]:
size(df_priests)

(4477, 28)

ermittle `df_idx` aus der CSV-Datei

In [283]:
df_item_utrecht = subset(df_item_db, :item_type_id => ByRow(isequal(item_type_id)));

In [284]:
size(df_item_utrecht, 1)

3958

In [27]:
df_idx = df_item_utrecht[:, [:id, :id_in_source, :item_type_id]];

In [28]:
names(df_priests)

25-element Vector{String}:
 "Bron"
 "Type record"
 "Priester_ID"
 "Datum (YMD)"
 "Locatie"
 "Transcriptie"
 "Voetnoot_bron"
 "Bijdrage"
 "Titel"
 "Voornaam"
 "Patroniem"
 "Familienaam"
 "Familienaam, toponiem"
 "Familienaam, onzeker"
 "Toponiem"
 "Orde"
 "Functie"
 "Instelling"
 "Voornaam_norm"
 "Patroniem_norm"
 "Orde_norm"
 "Opmerkingen"
 "Voornaam_uniciteit"
 "Patroniem_uniciteit"
 "Naam_ARDOU"

In [32]:
columns_bk = [
    :Priester_ID => :id_in_source,
    Symbol("Datum") => :ordination_date,
    Symbol("Ort (Locatie)") => :ordination_place,
    :Titel => :title,
    Symbol("Vorname (Voornaam)") => :givenname,
    Symbol("Zweiter Vorname (Patroniem)") => :patronym,
    Symbol("(Familienname) Familienaam") => :familyname,
    Symbol("Herkunftsort (Toponiem)") => :place_of_origin,        
    Symbol("Orden (Orde)") => :religious_order,
    Symbol("Amt (Functie)") => :role_name,
    :Instelling => :institution
];

In [22]:
names(df_priests)

28-element Vector{String}:
 "Bron"
 "Seite"
 "Type record"
 "Priester_ID"
 "Datum (YMD)"
 "Locatie"
 "Transcriptie"
 "Voetnoot_bron"
 "Bijdrage"
 "Titel"
 "Voornaam"
 "Patroniem"
 "Familienaam"
 ⋮
 "Orde"
 "Functie"
 "Instelling"
 "Voornaam_norm"
 "Patroniem_norm"
 "Orde_norm"
 "Opmerkingen"
 "Voornaam_uniciteit"
 "Patroniem_uniciteit"
 "Column26"
 "Column27"
 "Column28"

In [26]:
columns = [
    :Priester_ID => :id_in_source,
    :Transcriptie => :item_in_source,
    :Seite => :page,
    Symbol("Datum (YMD)") => :ordination_date,
    Symbol("Locatie") => :ordination_place,
    :Bijdrage => :contribution,
    :Titel => :title,    
    Symbol("Voornaam") => :givenname,
    Symbol("Patroniem") => :patronym,
    Symbol("Familienaam") => :familyname,
    Symbol("Familienaam, toponiem") => :familyname_t,
    Symbol("Familienaam, onzeker") => :familyname_o,
    Symbol("Toponiem") => :place_of_origin,        
    Symbol("Orde") => :religious_order,
    Symbol("Functie") => :role_name,
    :Bron => :reference,
    :Instelling => :institution
];

In [27]:
df_person_in = innerjoin(
    select(df_priests, columns),
    df_idx,
    on = :id_in_source,
    matchmissing = :notequal,
);

In [28]:
size(df_person_in, 1), size(df_priests, 1), size(df_idx, 1)

(4476, 4477, 4476)

### Quellen für Familiennamen

In [25]:
count_not_missing(cols...) = count(!ismissing, cols)

count_not_missing (generic function with 1 method)

In [37]:
transform!(df_person_in, [:familyname, :familyname_t, :familyname_o] => ByRow(count_not_missing) => :n_familynames);

In [38]:
df_info = subset(df_person_in, :n_familynames => ByRow((x -> x > 1)));

In [33]:
size(df_info)

(0, 19)

In [34]:
function select_not_missing(col_list...)
    val = missing
    for col in col_list
        if !ismissing(col)
            val = col
            break
        end
    end
    return val
end

select_not_missing (generic function with 1 method)

In [39]:
transform!(df_person_in, [:familyname, :familyname_t, :familyname_o] => ByRow(select_not_missing) => :familyname);

### Identifiziere Orden

In [ ]:
# Quelle: Server
file_name = joinpath(data_path, "religious_order.csv")
df_ro = CSV.read(file_name, DataFrame, delim = ";");
is_type_religious_order(id) = isequal(id, 7)
df_idx_ro = subset(df_item_db, :item_type_id => ByRow(is_type_religious_order);

In [90]:
# Quelle: lokale Datenbank
sql = "SELECT * FROM religious_order";
df_ro = Wds.sql_df(sql);
item_type_religious_order = 7;
sql = "SELECT id, id_in_source, item_type_id
FROM item
WHERE item_type_id = $(item_type_religious_order)";
df_idx_ro = Wds.sql_df(sql);

In [91]:
df_ro_id = innerjoin(df_ro, select(df_idx_ro, [:id, :item_type_id, :id_in_source]), on = :id);

Info

In [ ]:
df_ro_id

In [93]:
file_name = joinpath(data_path, "mapping_orden_stapel.csv")
df_map_ro = CSV.read(file_name, DataFrame, delim = ";");

In [94]:
transform!(df_map_ro, :id_order => ByRow(string) => :id_in_source);

In [95]:
df_map_ro = innerjoin(df_map_ro, df_ro_id, on = :id_in_source);

In [96]:
names(df_map_ro)

10-element Vector{String}:
 "altLabel_abk"
 "Orde"
 "id_order"
 "id_in_source"
 "id"
 "comment"
 "name"
 "abbreviation"
 "note"
 "item_type_id"

In [98]:
columns = [
    :id => :religious_order_id,
    :id_in_source => :id_in_source_ro,
    :Orde => :religious_order,
    :abbreviation => :abbreviation,
]

4-element Vector{Pair{Symbol, Symbol}}:
           :id => :religious_order_id
 :id_in_source => :id_in_source_ro
         :Orde => :religious_order
 :abbreviation => :abbreviation

In [99]:
df_map_ro_join = select(df_map_ro, columns);

Die Ordensbezeichnungen variieren in Groß- und Kleinschreibung

In [100]:
transform!(df_map_ro_join, :religious_order => ByRow(lowercase) => :religious_order);

In [101]:
df_map_ro_join[7:12, :]

,religious_order_id,id_in_source_ro,religious_order,abbreviation
,Int32,String,String,String?
1,5314,5,ordinis cistericiensis,OCist
2,5314,5,ordinis cistersiensium,OCist
3,176167,6,ordinis sancti wilhelmi,OESG
4,176167,6,ordinis wilhellemitarum,OESG
5,5319,12,ordinis sancti johannis,OMel
6,5309,13,ordinis predicatorum,OP


Wandle die Ordensbezeichnungen in Kleinbuchstaben um.

In [102]:
lowercase_maybe(a) = typeof(a) <: AbstractString ? lowercase(a) : a

lowercase_maybe (generic function with 1 method)

In [103]:
transform!(df_person_in, :religious_order => ByRow(lowercase_maybe) => :religious_order);

In [104]:
df_person_in_ro = leftjoin(df_person_in, df_map_ro_join, on = :religious_order, matchmissing = :notequal);

In [105]:
size(df_person_in, 1), size(df_person_in_ro, 1)

(4477, 4477)

In [106]:
count(!ismissing, df_person_in.religious_order), count(!ismissing, df_person_in_ro.religious_order_id)

(1359, 1352)

In [107]:
delta_msg(a, b) = !ismissing(a) && ismissing(b)

delta_msg (generic function with 1 method)

In [108]:
df_examine = subset(df_person_in_ro, [:religious_order, :religious_order_id] => ByRow(delta_msg));

In [109]:
unique(df_examine.religious_order)

1-element Vector{Union{Missing, String}}:
 "ordinis …"

Exportiere Daten für die Tabelle `person`

In [ ]:
names(df_person_in_ro)

Verwende das Jahr der Priesterweihe für die zeitliche Einordnung

In [118]:
parse_date_min(s) = ismissing(s) ? s : year(s)

parse_date_min (generic function with 1 method)

In [123]:
transform!(df_person_in_ro, :ordination_date => ByRow(parse_date_min) => :date_min);

In [124]:
columns = [
    :id => :id,
    :item_type_id => :item_type_id,
    :givenname => :givenname,
    :familyname => :familyname,
    :religious_order_id => :religious_order_id,
    :date_min => :date_min,
    :date_min => :date_max,
];

In [127]:
is_test(id) = id in (314332, 314338)

is_test (generic function with 1 method)

In [128]:
subset(df_person_in_ro[:, [:id, :givenname, :date_min, :ordination_date, :title]], :id => ByRow(is_test))

,id,givenname,date_min,ordination_date,title
,Int32,String?,Int64,Date,String?
1,314332,Laurencius,1511,1511-04-19,missing
2,314338,Bernardus,1511,1511-04-19,missing


In [129]:
table_name = "person"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, select(df_person_in_ro, columns))

┌ Info: row 
│   i = 2000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603
┌ Info: row 
│   i = 4000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603


4477

## Freie Attribute

### Titel

In [130]:
df_title = subset(df_person_in, :title => ByRow(!ismissing));

In [131]:
size(df_title)

(2105, 18)

In [132]:
df_item_prop_title = DataFrame();

In [133]:
insertcols!(df_item_prop_title,
    :item_id => df_title.id,
    :name => "title",
    :value => df_title.title,
    :date_value => missing,
);

In [134]:
df_item_prop_title[70:75, :]

,item_id,name,value,date_value
,Int32,String,String?,Missing
1,312487,title,Magister,missing
2,312489,title,Magister,missing
3,312493,title,Magister,missing
4,312494,title,Frater,missing
5,312495,title,Magister,missing
6,312496,title,Magister,missing


### Weihe Priester

Weiheamt

In [135]:
df_ordination = subset(df_person_in, :id => ByRow(!ismissing));
size(df_ordination)

(4477, 18)

In [136]:
df_item_prop_ord = DataFrame();

In [137]:
insertcols!(df_item_prop_ord,
    :item_id => df_ordination.id,
    :name => "ordination_priest",
    :value => "Priester",
    :date_value => Date.(df_ordination.ordination_date)
);

Ort der Weihe

In [138]:
df_ord_place = subset(df_person_in, :ordination_place => ByRow(!ismissing));
size(df_ord_place)

(4477, 18)

In [139]:
df_item_prop_ord_place = DataFrame();

In [140]:
insertcols!(df_item_prop_ord_place,
    :item_id => df_ord_place.id,
    :name => "ordination_priest_place",
    :value => df_ord_place.ordination_place,
    :date_value => missing,
);

Tabelle `item_property` schreiben

In [141]:
df_item_prop = vcat(df_item_prop_title, df_item_prop_ord, df_item_prop_ord_place);

In [142]:
df_item_prop[1712:1714, :]

,item_id,name,value,date_value
,Int32,String,String?,Date?
1,315892,title,Frater,missing
2,315894,title,Frater,missing
3,315895,title,Frater,missing


In [143]:
size(df_item_prop)

(11059, 4)

In [144]:
table_name = "item_property"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, df_item_prop)

┌ Info: row 
│   i = 2000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603
┌ Info: row 
│   i = 4000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603
┌ Info: row 
│   i = 6000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603
┌ Info: row 
│   i = 8000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603
┌ Info: row 
│   i = 10000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603


11059

### verminderter Beitrag

In [27]:
df_item_red_contrib = dropmissing(df_person_in, :contribution);

In [28]:
size(df_item_red_contrib)

(802, 19)

In [30]:
names(df_item_red_contrib)

19-element Vector{String}:
 "id_in_source"
 "item_in_source"
 "page"
 "ordination_date"
 "ordination_place"
 "contribution"
 "title"
 "givenname"
 "patronym"
 "familyname"
 "familyname_t"
 "familyname_o"
 "place_of_origin"
 "religious_order"
 "role_name"
 "reference"
 "institution"
 "id"
 "item_type_id"

In [39]:
df_item_prop_rc = select(df_item_red_contrib, [:id => :item_id]);

In [40]:
size(df_item_prop_rc)

(802, 1)

In [41]:
insertcols!(df_item_prop_rc,
    :name => "contribution",
    :value => "reduced"
);

In [42]:
table_name = "item_property"
file_name = joinpath(out_path, "$(table_name)_red_contrib.sql")
Wds.insert_sql(file_name, table_name, df_item_prop_rc)

802

### Geburtsort
*2022-04-12*: Trage zunächst nur den Namen ein. Die Zuordnung zu identifizierten Orten ist unklar.  
*2022-05-02*: siehe Notebook `Utrecht-Orte`. Es gibt eine eigene Tabelle `person_birthplace`.

In [145]:
names(df_person_in)

18-element Vector{String}:
 "id_in_source"
 "item_in_source"
 "ordination_date"
 "ordination_place"
 "title"
 "givenname"
 "patronym"
 "familyname"
 "familyname_t"
 "familyname_o"
 "place_of_origin"
 "religious_order"
 "role_name"
 "reference"
 "institution"
 "id"
 "item_type_id"
 "n_familynames"

In [146]:
df_birthplace = subset(df_person_in, :place_of_origin => ByRow(!ismissing));
size(df_birthplace)

(4024, 18)

In [147]:
df_item_prop_birthplace = DataFrame();

In [148]:
insertcols!(df_item_prop_birthplace,
    :item_id => df_birthplace.id,
    :name => "birthplace",
    :value => df_birthplace.place_of_origin,
);

In [ ]:
table_name = "item_property"
file_name = joinpath(out_path, "$(table_name)_birthplace.sql")
Wds.insert_sql(file_name, table_name, df_item_prop_birthplace)

## Literaturverweise eintragen
*R bk* Befülle `id_in_reference` mit 'id_1509_3_83' ... ?

In [29]:
df_item_reference_1 = DataFrame();

In [30]:
insertcols!(df_item_reference_1,
    :reference_id => 1,
    :page => missing,
    :id_in_reference => df_person_in.id_in_source,
    :item_id => df_person_in.id,
    :item_type_id => item_type_id
);

In [31]:
df_item_reference_1[1500:1505, :]

,reference_id,page,id_in_reference,item_id,item_type_id
,Int64,Missing,String,Int32,Int64
1,1,missing,id_1509_5_64,313827,10
2,1,missing,id_1509_5_65,313828,10
3,1,missing,id_1509_5_66,313829,10
4,1,missing,id_1509_5_67,313830,10
5,1,missing,id_1509_5_68,313831,10
6,1,missing,id_1509_5_69,313832,10


In [32]:
columns = [
    :id => :item_id,
    :page => :page,
]

2-element Vector{Pair{Symbol, Symbol}}:
   :id => :item_id
 :page => :page

In [33]:
df_item_reference_2 = select(df_person_in, columns);

In [35]:
insertcols!(df_item_reference_2,
    :reference_id => missing,
    :item_type_id => 10,
    :id_in_reference => missing,
);

In [38]:
select_reference(page) = ismissing(page) ? missing : (page <= 85 ? 3 : 2)

select_reference (generic function with 1 method)

In [42]:
transform!(df_item_reference_2, :page => ByRow(select_reference) => :reference_id);

In [43]:
df_item_reference = vcat(df_item_reference_1, df_item_reference_2);

In [44]:
table_name = "item_reference"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, df_item_reference)

┌ Info: row 
│   i = 2000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603
┌ Info: row 
│   i = 4000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603
┌ Info: row 
│   i = 6000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603
┌ Info: row 
│   i = 8000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603


8952

## Namensvarianten eintragen
Schreibe Patronym (Vatersnamen) in Tabelle `item_property`

In [171]:
df_patronym = subset(df_person_in, :patronym => ByRow(!ismissing));

In [172]:
df_patronym[70:75, [:id, :givenname, :patronym]]

,id,givenname,patronym
,Int32,String?,String?
1,312444,Franciscus,Ghysberti
2,312445,Ancelinus,Johannis
3,312446,Jacobus,Johannis
4,312447,Johannes,Hermanni
5,312448,Wilhelmus,Michaelis
6,312449,Johannes,Peregrini


In [173]:
df_item_prop_pym = DataFrame()

""
""


In [174]:
insertcols!(df_item_prop_pym,
    :item_id => df_patronym.id,
    :name => "patronym",
    :value => df_patronym.patronym
);

In [175]:
df_item_prop_pym[70:75, :]

,item_id,name,value
,Int32,String,String?
1,312444,patronym,Ghysberti
2,312445,patronym,Johannis
3,312446,patronym,Johannis
4,312447,patronym,Hermanni
5,312448,patronym,Michaelis
6,312449,patronym,Peregrini


In [176]:
table_name = "item_property"
file_name = joinpath(out_path, "$(table_name)_patronym.sql")
Wds.insert_sql(file_name, table_name, df_item_prop_pym)

┌ Info: row 
│   i = 2000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603


2984

### Suchvarianten eintragen in `name_lookup`
Das Patronym entspricht für die Suche am ehesten einem zweiten Vornamen.

In [40]:
df_person_name = select(df_person_in, [:id, :givenname, :patronym, :familyname]);

In [41]:
function join_space(a...)
    c_a = filter(!ismissing, collect(a))
    return join(c_a, " ")
end

join_space (generic function with 1 method)

In [42]:
df_name_variant = transform(df_person_name, [:givenname, :patronym] => ByRow(join_space) => :givenname);

In [43]:
df_name_variant[50:55, :]

,id,givenname,patronym,familyname
,Int32,String,String?,String?
1,312376,Heinricus Cornelii,Cornelii,missing
2,312377,Nicolaus Petri,Petri,missing
3,312378,Johannes Mathei,Mathei,missing
4,312379,Hermannus,missing,missing
5,312380,Jordanus,missing,missing
6,312381,Cornelis Johannes,Johannes,missing


Füge technisch erforderliche Spalten ein. Erzeuge Einträge für `name_lookup`.

In [44]:
insertcols!(df_name_variant,
    :givenname_variant => missing,
    :familyname_variant => missing,
    :prefix_name => missing,
    :lang => "nl"
);

In [45]:
df_nl = Wds.create_name_lookup(df_name_variant);

In [46]:
df_nl[56:60, :]

,person_id,gn_fn,gn_prefix_fn
,Int64,String?,String?
1,312363,Judocus Gerardi,missing
2,312363,Judocus,missing
3,312364,Johannes Jacobi,missing
4,312364,Johannes,missing
5,312365,Amelius Laurencii,missing


In [186]:
names(df_nl)

3-element Vector{String}:
 "person_id"
 "gn_fn"
 "gn_prefix_fn"

In [47]:
table_name = "name_lookup"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, df_nl)

┌ Info: row 
│   i = 2000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603
┌ Info: row 
│   i = 4000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603
┌ Info: row 
│   i = 6000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1603


7461

## Ämter

In [193]:
# Quelle: Datenbank lokal
sql = "SELECT id, id_in_source, item_type_id
FROM item 
WHERE item_type_id = 10";
df_idx = Wds.sql_df(sql);

In [194]:
size(df_idx)

(4477, 3)

*2022-05-25* lies Daten aus `Ordinations_of_priests_Stapel_BK.csv`, weil hier die Ämter übertragen sind

In [228]:
file_name = joinpath(data_path, "Ordinations_of_priests_Stapel_BK.csv");
df_priests = CSV.read(file_name, DataFrame, delim = ';', stringtype = String);

In [229]:
Wds.clean_up!(df_priests)

In [230]:
dropmissing!(df_priests, :Priester_ID);

In [231]:
df_priests = unique(df_priests);

In [232]:
size(df_priests, 1)

3958

In [234]:
columns_bk = [
    :Priester_ID => :id_in_source,
    :Datum => :ordination_date,
    Symbol("Ort (Locatie)") => :ordination_place,
    :Titel => :title,
    Symbol("Vorname (Voornaam)") => :givenname,
    Symbol("Zweiter Vorname (Patroniem)") => :patronym,
    Symbol("(Familienname) Familienaam") => :familyname,
    Symbol("Herkunftsort (Toponiem)") => :place_of_origin,        
    Symbol("Orden (Orde)") => :religious_order,
    Symbol("Amt (Functie)") => :role_name,
    :Instelling => :institution
];

In [235]:
df_person_in = innerjoin(
    select(df_priests, columns_bk),
    df_idx,
    on = :id_in_source,
    matchmissing = :notequal,
);

In [236]:
df_person_role = dropmissing(df_person_in, :role_name);

In [188]:
# Quelle: Server
file_name = joinpath(data_path, "item.csv");
df_item_db = CSV.read(file_name, DataFrame, delim = ';');

In [189]:
file_name = joinpath(data_path, "Ordinations_of_priests_Stapel_BK.csv");
df_priests = CSV.read(file_name, DataFrame, delim = ';', stringtype = String);

In [190]:
Wds.clean_up!(df_priests)

In [191]:
size(df_priests, 1)

5255

In [192]:
df_priests = unique(df_priests);

In [22]:
df_item_utrecht = subset(df_item_db, :item_type_id => ByRow(isequal(10)));

In [23]:
size(df_item_utrecht, 1)

3958

In [24]:
columns = [
    :id => :person_id,
    :id_in_source => :id_in_source,
    :item_type_id => :item_type_id,
]

3-element Vector{Pair{Symbol, Symbol}}:
           :id => :person_id
 :id_in_source => :id_in_source
 :item_type_id => :item_type_id

In [25]:
df_idx = select(df_item_utrecht, columns);

In [212]:
columns_bk = [
    :Priester_ID => :id_in_source,
    :Datum => :ordination_date,
    Symbol("Ort (Locatie)") => :ordination_place,
    :Titel => :title,
    Symbol("Vorname (Voornaam)") => :givenname,
    Symbol("Zweiter Vorname (Patroniem)") => :patronym,
    Symbol("(Familienname) Familienaam") => :familyname,
    Symbol("Herkunftsort (Toponiem)") => :place_of_origin,        
    Symbol("Orden (Orde)") => :religious_order,
    Symbol("Amt (Functie)") => :role_name,
    :Instelling => :institution
];

In [ ]:
df_priests_with_role = subset(select(df_person_in, columns_bk), :role_name => ByRow(!ismissing));

In [28]:
df_person_role = innerjoin(
    df_priests_with_role,
    df_idx,
    on = :id_in_source
);

In [216]:
size(df_person_role, 1), size(df_priests, 1), size(df_idx, 1)

(57, 4477, 4477)

Setze Diözese 'Utrecht', ID = 53

In [237]:
insertcols!(df_person_role, :diocese_id => 53, :diocese_name => "Utrecht");

In [238]:
df_person_role[30:35, [:id_in_source, :givenname, :familyname, :role_name, :diocese_name, :diocese_id]]

,id_in_source,givenname,familyname,role_name,diocese_name,diocese_id
,String,String,String?,String,String,Int64
1,id_1514_1_2,Petrus,missing,Vikar,Utrecht,53
2,id_1514_2_1,Cornelius,missing,Vikar,Utrecht,53
3,id_1514_3_1,Petrus,Magistri,Vikar,Utrecht,53
4,id_1514_3_2,Johannes,missing,Vikar,Utrecht,53
5,id_1514_4_1,Johannes,missing,Kanoniker,Utrecht,53
6,id_1515_2_1,Wilhelmus,missing,Vikar,Utrecht,53


### Rollen zuordnen

In [34]:
Wds.setDBWIAG(user = "georg", db = "wiag2")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2")

In [239]:
# Quelle Datenbank
sql = "SELECT * FROM role";
df_role = Wds.sql_df(sql);

In [32]:
# Quelle csv
file_name = joinpath(data_path, "role.csv");
df_role = CSV.read(file_name, DataFrame);

In [220]:
names(df_role)

4-element Vector{String}:
 "id"
 "note"
 "name"
 "comment"

In [240]:
df_person_role[21:25, [:id, :givenname, :institution, :diocese_id, :diocese_name, :role_name]]

,id,givenname,institution,diocese_id,diocese_name,role_name
,Int32,String,String?,Int64,String,String
1,313877,Jacobus,Verensis,53,Utrecht,Kanoniker
2,313957,Johannes,Werden,53,Utrecht,familiaris
3,314042,Jacobus,Trajectensis,53,Utrecht,Vikar
4,314093,Lodewicus,Wijck,53,Utrecht,Kanoniker
5,314279,Bruno,ecclesie sancte Marie Trajectensis,53,Utrecht,Vikar


In [241]:
df_person_role = leftjoin(df_person_role, 
    select(df_role, [:id => :role_id, :name => :role_name_norm]), 
    on = :role_name => :role_name_norm);

In [242]:
count(ismissing, df_person_role.role_id)

2

In [39]:
columns = [
    :person_id, :id_in_source, :title, :givenname, :patronym, :familyname, :role_name, :role_id
];

In [40]:
subset(select(df_person_role, columns), :role_id => ByRow(ismissing))

,person_id,id_in_source,title,givenname,patronym,familyname,role_name,role_id
,Int64,String,String?,String,String?,String?,String?,Int32?
1,180550,id_1507_3_4,missing,Gerardus,Jacobi,missing,Vikar etc.,missing
2,183986,id_1510_3_1,missing,Johannes,missing,Nayven,familiaris,missing


In [246]:
df_person_role[26:30, [:id, :id_in_source, :givenname, :role_name, :role_id]]

,id,id_in_source,givenname,role_name,role_id
,Int32,String,String,String,Int32?
1,314042,id_1510_4_2,Jacobus,Vikar,249238
2,314279,id_1511_3_1,Bruno,Vikar,249238
3,314558,id_1512_2_1,Petrus,Vikar,249238
4,314610,id_1512_3_1,Gherlacus,Vikar,249238
5,314838,id_1512_6_1,Ernestus,Vikar,249238


In [245]:
names(df_person_role)

16-element Vector{String}:
 "id_in_source"
 "ordination_date"
 "ordination_place"
 "title"
 "givenname"
 "patronym"
 "familyname"
 "place_of_origin"
 "religious_order"
 "role_name"
 "institution"
 "id"
 "item_type_id"
 "diocese_id"
 "diocese_name"
 "role_id"

In [247]:
columns = [
    :id => :person_id,
    :role_id => :role_id,
    :role_name => :role_name,
    :diocese_id => :diocese_id,
    :diocese_name => :diocese_name,
    :institution => :institution_name,
];

In [248]:
df_person_role_out = select(df_person_role, columns);

In [249]:
# Ausgabe in Datei
table_name = "person_role"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, df_person_role_out)

54

In [250]:
df_person_role_out[21:25, :]

,person_id,role_id,role_name,diocese_id,diocese_name,institution_name
,Int32,Int32?,String,Int64,String,String?
1,313478,249238,Vikar,53,Utrecht,ecclesie Trajectensis
2,313549,249238,Vikar,53,Utrecht,ecclesie sancti Johannis
3,313583,249238,Vikar,53,Utrecht,ecclesie Trajectensis
4,313625,249238,Vikar,53,Utrecht,ecclesie sancti Petri
5,313685,249238,Vikar,53,Utrecht,ecclesie Trajectensis


In [46]:
# Ausgabe in die Datenbank
table_name = "person_role"
Wds.filltable!(table_name, df_person_role_out)

┌ Info: Rows inserted: 54
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1209


54

Dateien zusammenfügen

In [ ]:
function cat_sql(file_out, file_in...; path = ".")    
    open(joinpath(path, file_out), "w") do io_out
        for file_loop in file_in            
            open(joinpath(path, file_loop)) do io_in
                write(io_out, read(io_in))
            end
        end
    end
end

Inhalte, die unabhängig sind von einzelnen Personen: `item_type.sql`, `reference_volume.sql`

Inhalte für die einzelnen Priester. (Tabelle `item` muss schon im Prozess geschrieben und gelesen worden sein).

In [ ]:
file_out = "item_x.sql"
cat_sql(file_out,
    "item_property.sql",
    "item_property_patronym.sql",
    "item_reference.sql",
    path = out_path,
)

In [ ]:
file_out = "person_x.sql"
cat_sql(file_out,
    "person.sql",
    "person_role.sql",
    "name_lookup.sql",
    path = out_path,
)